In [ ]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from tensorflow import keras

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Embedding, Flatten

import numpy
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, Concatenate, Dropout, Attention
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# tf.config.set_visible_devices(gpus[:1], 'GPU')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Available GPUs: {len(gpus)}")
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu}")
else:
    print("No GPUs available.")    

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
class Attention(tf.keras.Model):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.w1 = Dense(units)
        self.w2 = Dense(units)
        self.V = Dense(1)
        
    def __call__(self, values, query):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.w1(values) + self.w2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
            
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
            
        return context_vector, attention_weights

# 파라미터 튜닝 결과 모든 모델에서 1개가 최적이었으므로 노드 수만 조절
def DDI_model(unit):
    input_A = layers.Input(shape=(train_A.shape[1], 700), name = 'inputA')
    input_B = layers.Input(shape=(train_A.shape[1], 700), name = 'inputB')

    lstm_A, forward_h_A, forward_c_A, backward_h_A, backward_c_A = Bidirectional(LSTM(64, return_sequences=True, return_state = True), name = "Bi_A")(input_A)
    lstm_B, forward_h_B, forward_c_B, backward_h_B, backward_c_B = Bidirectional(LSTM(64, return_sequences=True, return_state = True), name = "Bi_B")(input_B)

    state_h_A = Concatenate()([forward_h_A, backward_h_A])
    state_h_B = Concatenate()([forward_h_B, backward_h_B])

    attention_A = Attention(80)
    context_vector_A, attention_weight_A = attention_A(lstm_A, state_h_A)

    attention_B= Attention(80)
    context_vector_B, attention_weight_B = attention_B(lstm_B, state_h_B)

    concat_AB = Concatenate()([context_vector_A, context_vector_B])

    fc = Dense(units = unit, activation = 'relu', kernel_initializer = 'he_normal')(concat_AB)

    output = Dense(units = num_class, activation = 'softmax')(fc)

    model = Model(inputs = [input_A,input_B], outputs = output)
    
    return model
    
# 데이터 로드
train_set = np.load('../Generated_data/train_set.npz')
val_set = np.load('../Generated_data/val_set.npz')

train_A = train_set['train_A']
train_B = train_set['train_B']

val_A = val_set['val_A']
val_B = val_set['val_B']
one_hot_train = pd.read_csv('../Generated_data/one_hot_train.csv', index_col = 0)
one_hot_val = pd.read_csv('../Generated_data/one_hot_val.csv', index_col = 0)

num_class = len(one_hot_train.columns)
del train_set, val_set

es = EarlyStopping(patience = 10, restore_best_weights = True)
mc = ModelCheckpoint("Model_save/model_training.h5", save_best_only = True)

model = DDI_model(128)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.AUC(name='auc')])

model.fit({'inputA':train_A, 'inputB':train_B}, one_hot_train, 
          epochs = 100, 
          batch_size = 32, 
          verbose = 2, 
          validation_data=({'inputA':val_A, 'inputB':val_B}, one_hot_val),
          callbacks = [es, mc])

model.save('Model_save/model_final.h5', include_optimizer=False)